# Convert CSV to Parquet

In [10]:
import polars as pl
from_path = 'D:/data/dsio'
to_path = 'C:/Users/USER/PycharmProjects/research/data'

(
    pl.scan_csv(f'{from_path}/tb_dyn_fcst_dmnd.csv'
                , has_header=True
                , infer_schema_length = 100_000
                , dtypes = {'CONT_AMT': pl.Float64})
 .collect().write_parquet(f"{to_path}/tb_dyn_fcst_dmnd.parquet")
)

In [12]:
(
    pl.scan_csv(f'{from_path}/tb_dyn_fcst_dmnd_sellout.csv'
                , has_header = True
                , infer_schema_length = 100_000
                , dtypes = {'CONT_AMT': pl.Float64})
    .collect().write_parquet(f"{to_path}/tb_dyn_fcst_dmnd_sellout.parquet")
)

In [13]:
(
    pl.scan_csv(f'{from_path}/tb_bas_oper_part_mst.csv'
                , has_header = True
                , infer_schema_length = 100_000
                , dtypes = {'CONT_AMT': pl.Float64})
    .collect().write_parquet(f"{to_path}/tb_bas_oper_part_mst.parquet")
)

In [4]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.version.cuda)
print(torch.__version__)
print(torch.cuda.get_device_name(0))
print(torch.__version__)

False
0
None
2.6.0+cpu


AssertionError: Torch not compiled with CUDA enabled

In [3]:
import sys
print(sys.executable)

C:\Users\USER\python\py312\python.exe
